In [ ]:
# ==================== 环境设置和库导入 ====================
import sys
from pathlib import Path
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# 设置项目路径
project_root = Path("..").resolve()
if str(project_root / "src") not in sys.path:
    sys.path.append(str(project_root / "src"))

# 导入 echemistpy
from echemistpy.io.loaders import load
from echemistpy.processing.analyzers.stxm import STXMAnalyzer

print(f"✓ 项目根目录: {project_root}")
print("✓ 环境配置完成!")


# 🧪 STXMAnalyzer 测试 Notebook

使用真实的 TXM HDF5 数据测试 `STXMAnalyzer`，包括 PCA 降噪、聚类分析和光谱拟合。


In [ ]:
# 设置数据文件路径
data_file = project_root / "docs" / "examples" / "TXM" / "αMnO2" / "20230629_E1A_749.7x177.5y_specnorm_aliOF.hdf5.hdf5"

# 加载数据
raw_data, raw_info = load(data_file)

print(f"✓ 数据加载成功")
print(f"\n数据维度: {dict(raw_data.data.sizes)}")
print(f"数据变量: {list(raw_data.data.data_vars)}")
print(f"\n样品信息:")
print(f"  - 样品名: {raw_info.sample_name}")
print(f"  - 仪器: {raw_info.instrument}")
print(f"  - 技术: {raw_info.technique}")


## 2. 执行分析

使用 `STXMAnalyzer` 进行 PCA 降噪、聚类分析和光谱拟合。


In [ ]:
# 创建分析器并执行分析
analyzer = STXMAnalyzer(
    pca_components=6,
    perform_clustering=True,
    n_clusters=4,
    perform_fitting=True,
)

result_data, result_info = analyzer.analyze(raw_data, raw_info)

# 查看结果
print("✓ 分析完成\n")
print(f"结果参数: {result_info.parameters}\n")
print(f"结果数据集:\n{result_data.data}")


## 3. 可视化结果

绘制降噪图像、聚类标签和聚类光谱。


In [ ]:
ds_res = result_data.data

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. 降噪后的总强度图
if "denoised" in ds_res and "energy_eV" in ds_res.denoised.dims:
    img_sum = ds_res.denoised.sum(dim="energy_eV")
    im0 = axes[0].imshow(img_sum, cmap="viridis")
    axes[0].set_title("PCA 降噪后总强度")
    plt.colorbar(im0, ax=axes[0])
else:
    axes[0].text(0.5, 0.5, "无降噪数据", ha='center', va='center')
    axes[0].set_title("PCA 降噪")

# 2. 聚类标签图
if "cluster_labels" in ds_res:
    im1 = axes[1].imshow(ds_res.cluster_labels, cmap="tab10")
    axes[1].set_title(f"聚类标签 (K={analyzer.n_clusters})")
    plt.colorbar(im1, ax=axes[1], ticks=range(analyzer.n_clusters))
else:
    axes[1].text(0.5, 0.5, "无聚类数据", ha='center', va='center')
    axes[1].set_title("聚类标签")

# 3. 聚类光谱
if hasattr(result_info, "others") and "cluster_centroids" in result_info.others:
    centroids = result_info.others["cluster_centroids"]
    energy = ds_res.energy_eV.values
    for i, spectrum in enumerate(centroids):
        axes[2].plot(energy, spectrum, label=f"簇 {i}", linewidth=2)
    axes[2].set_xlabel("能量 (eV)")
    axes[2].set_ylabel("光学密度")
    axes[2].set_title("聚类中心光谱")
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
else:
    axes[2].text(0.5, 0.5, "无光谱数据", ha='center', va='center')
    axes[2].set_title("聚类光谱")

plt.tight_layout()
plt.show()
